In [11]:
import json
import re
import numpy as np 
import pandas as pd

In [12]:
import importlib.util as imp
import sys

spec = imp.spec_from_file_location(
    'twitter_connection', 
    '../twitter-connection/__init__.py')
twit = imp.module_from_spec(spec)
sys.modules[spec.name] = twit
spec.loader.exec_module(twit)

In [13]:
'''
File path to the bearer token. Requires a prefix to identify the
  token, which is just 'PERSONAL $BEARER_TOKEN$' by default -- 
  can be specified upon initialization of TwitterConnection
'''
cred_path = r'../twitter-connection/credentials.txt'

with open('../extraction/verb-stem-clean.txt') as f:
    verb_stem = json.load(f)

query_cond = '(lang:es OR lang:pt) has:geo -is:retweet -has:links '
fields_tweet = 'tweet.fields=lang,geo,created_at,public_metrics,referenced_tweets'
fields_expan = 'expansions=author_id,geo.place_id,entities.mentions.username'
fields_user = 'user.fields=created_at,location,public_metrics'
fields_place = 'place.fields=country'

conn = twit.TwitterConnection(
    is_archive=True,
    cred_prefix='PROF')

conn.set_query(conditions=query_cond)
conn.set_fields(tweet=fields_tweet, 
                      expansions=fields_expan, 
                      user=fields_user,
                      place=fields_place)

s = None
for vs in verb_stem.items():
    s = vs[1]
    break

s = '(' + s + ')'
s

conn.connect(s, is_next=True)

twit.response.save('test-data.txt', conn.response)

In [14]:
table = twit.response.retrieve('test-data.txt')

In [15]:
table.keys()

dict_keys(['data', 'includes', 'meta'])

In [16]:
table['includes'].keys()

dict_keys(['users', 'places'])

In [17]:
data = pd.json_normalize(table['data'])
users = pd.json_normalize(table['includes'], record_path='users')
places = pd.json_normalize(table['includes'], record_path='places')

display(data.head(3))
display(users.head(3))
display(places.head(3))

,lang,created_at,text,id,author_id,geo.place_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,entities.mentions,referenced_tweets
0,pt,2021-06-27T15:29:11.000Z,"Acordei agora, fui no site vê a camisa nova e ...",1409171960337158148,1268193349980303360,edca241a7659fad8,0,0,0,0,NaN,NaN
1,pt,2021-06-27T15:29:03.000Z,Leal escolheu o Brasil como pátria por uma que...,1409171927713955844,35773229,0090a6f53f20ebd0,0,0,0,0,NaN,NaN
2,es,2021-06-27T15:28:52.000Z,@Nostradamus_17 \nTengo poco siguiéndote pero ...,1409171882214105097,560766195,b8779fa4f42b81de,0,0,0,0,"[{'start': 0, 'end': 15, 'username': 'Nostrada...",NaN


,name,id,username,location,created_at,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,érica rocha ✰,1268193349980303360,euerica_crvg,Plataforma 9 ¾ Hogwarts,2020-06-03T14:50:47.000Z,98,140,6674,0
1,DALMO CARVALHO,35773229,DALMOCARVALHO,Brasil,2009-04-27T15:42:35.000Z,21,9,828,0
2,...evil...,560766195,Dansachio,NaN,2012-04-22T23:52:57.000Z,255,946,7885,0


,country,full_name,id
0,Brazil,"Rio das Ostras, Brasil",edca241a7659fad8
1,Brazil,"São Bernardo do Campo, Brasil",0090a6f53f20ebd0
2,Mexico,"Victoria, Tamaulipas",b8779fa4f42b81de


In [23]:
from unidecode import unidecode

In [38]:
# Remove '@...' mentions
data.loc[:, 'text'] = data.loc[:, 'text'].str.replace(r'(@[\w]+ )', '', regex=True)

# Duplicated text
text = data.loc[:, 'text'].apply(unidecode)
dup = data.loc[:, 'text'].duplicated()
topic = ~(text.str.contains(
    r'\bve|\bvi', flags=re.IGNORECASE, regex=True))

print(f'\n{dup.sum()} duplicated, {topic.sum()} bad matches')
display(
    pd.concat([data.loc[:, 'text'], 
               dup.rename('is_dup'), 
               ~(topic.rename('no_stem'))], 
              axis=1))

for t in text.loc[~topic]:
    n = t.lower().replace('ve', '|>|ve|<|').replace('vi', '|>|vi|<|')
    print(f'CASE:\n{n}\n')


0 duplicated, 0 bad matches


,text,is_dup,no_stem
0,"Acordei agora, fui no site vê a camisa nova e ...",False,True
1,Leal escolheu o Brasil como pátria por uma que...,False,True
2,\nTengo poco siguiéndote pero se ve reflejado ...,False,True
3,Manes es el Massa de este lado. Sin las fotos ...,False,True
4,"Es q el comunismo al q tanto temes, está solo ...",False,True
...,...,...,...
95,Te entiendo pero lo primero que se ve es la fo...,False,True
96,Aí quando vê já tá tarde pra dar banho 🤣,False,True
97,Faltando 3 minutos pra chegar no destino thyel...,False,True
98,Vê se pode,False,True


CASE:
acordei agora, fui no site |>|ve|<| a camisa nova e ja esgotou. minha torcida faz tudo. aqui e vasco!

CASE:
leal escolheu o brasil como patria por uma questao estrategica... |>|ve|<| se ele iria escolher uruguai claro que nao! nao escolheu a patria escolheu a con|>|ve|<|niencia #voleinosportv

CASE:

tengo poco siguiendote pero se |>|ve|<| reflejado tu trabajo, una mala racha no lo define animo, gracias por compartir tus picks...

CASE:
manes es el massa de este lado. sin las fotos ridiculas con la esposa pero tengan paciencia porque quien te a |>|vi|<|sto y quien te |>|ve|<|.

CASE:
es q el comunismo al q tanto temes, esta solo en tu mente...
nadie quiere imppner ese tipo d gobierno en chile, solo se busca frenar el.abuso, parar a los corruptos, sean d terno, d sotana o.d.uniforme, q no haya lucro en educacion y salud publicas, y mejorar las pensiones.
|>|ve|<|?

CASE:
muito bom ne, eu |>|vi|<| so metade pq eu dormi kkk mas achei mt legal

CASE:
simmm hahahahaha descobri ontem.

In [ ]:
mod = data.drop(
    data.loc[dup|(~topic)].index)

print(f'\nAfter dropping: {mod.shape[0]}')
display(mod.head())

In [142]:
topic = n.str.contains(
    r'\bve|\bvi', flags=re.IGNORECASE, regex=True)
dup = places.loc[:, 'id'].duplicated()

matching = places.loc[dup]
print(f'Extracted: {places.shape[0]}, matching: {matching.shape[0]}')

Extracted: 76, matching: 0


In [144]:
places.describe()

,country,full_name,id
count,76,76,76
unique,16,76,76
top,Brazil,"Rio de Janeiro, Brazil",01fd50ae72f37614
freq,32,1,1


In [139]:
for t in matching:
    
    found = t.lower().replace('ve', '|>|ve|<|').replace('vi', '|>|vi|<|')
    print(f'\nCASE:\n{found}')


CASE:
@arisquetaigea en colonia al salir |>|vi|<| cada 3 puertas la misma información: elektronischer bla bla solo en la farmacia. preguntaré el lunes a |>|ve|<|r... gracias!

CASE:
@_eduardam28 short e top?? |>|vi|<| ontem e fiquei loucaaaa

CASE:
#porquesomosderecha, porque conoci de primera mano el accionar del comunismo. porque |>|vi|<| a mi chile renacer de las cenizas, en que lo dejaron las politicas progresista.
porque quiero q mis hijos |>|vi|<|van en en pais libre y prospero
porque amo a mi chile

CASE:
@lecralencar quem vê até pensa

CASE:
datos q enamoran después d los 30’: 
* t duele algo?
* si comes tanto no vas a poder dormir! 
* descansa para salir en la noche! 
* y si pedimos comida y comemos en casa no más? 
* este juego d mesa se |>|ve|<| che|>|ve|<|re. compremos! 
* de q tiene hambre?
* y así… 
solo para entendidos!

CASE:
já |>|vi|<| neguin falar que é braço e abandonar a missão

CASE:
@wondymaximoff é cada declaração que a gente vê... meu deus

CASE:
não |>|vi|<| 